In [15]:
# N1 : ImageNet | Food-101 | Rice dataset

In [16]:
## imports
import os
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    device = 'cuda'

CUDA is available!  Training on GPU ...


In [18]:
## function to train the network
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss /= len(train_loader.dataset)
    print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch, correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset), train_loss))

## function to train the network
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test loss: {:.4f}, Test Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
data_transform =  transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [20]:
model = models.vgg19(pretrained=True)
#print(model)

### FOOD-101

In [21]:


data_dir = '/floyd/input/food_101/images'
batch_size = 100
num_workers = 0
dataset = datasets.ImageFolder(
        data_dir, 
        transform=data_transform)
print(len(dataset),"images")
## split data between train and test. Total of 101,000 images 
## we use 100,000 for aining and 1,000 for testing
train_data,test_data=torch.utils.data.random_split(dataset,(100000,1000))
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)



101000 images


In [22]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [23]:
import torch.nn as nn

n_classes = 101 ## Number of classes in our rice dataset

n_inputs = model.classifier[-1].in_features
last_layer = nn.Linear(n_inputs,n_classes)
model.classifier[-1] = last_layer
# if GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()
# check to see that your last layer produces the expected number of outputs
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=101, bias=True)
)


In [24]:
import torch.optim as optim
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [25]:
# number of epochs to train the model
n_epochs = 3

print ("Start training")

for epoch in range(1, n_epochs+1):
    
    train (model, device, train_loader, optimizer, criterion, epoch)
    test (model, device, criterion, test_loader)
    
    print ('\n')
            
print ("Done training")

Start training
Epoch: 1 , Training Accuracy: 27216/100000 (27%) Training Loss: 0.031090
Test loss: 0.0242, Test Accuracy: 424/1000 (42%)



Epoch: 2 , Training Accuracy: 34370/100000 (34%) Training Loss: 0.027749
Test loss: 0.0241, Test Accuracy: 450/1000 (45%)



Epoch: 3 , Training Accuracy: 37867/100000 (38%) Training Loss: 0.026235
Test loss: 0.0230, Test Accuracy: 462/1000 (46%)



Done training


In [26]:
torch.save(model.state_dict(), 'food-101-trained.pwf')

### RICE DATASET

In [14]:

data_dir = '/floyd/input/rice_dataset/'
batch_size = 100
num_workers = 0


train_data = datasets.ImageFolder(
        data_dir+'training_data', 
        transform=data_transform)

test_data = datasets.ImageFolder(
        data_dir+'test_data', 
        transform=data_transform)


print(len(train_data)+len(test_data),"images")
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)


428 images


In [15]:
import torch.nn as nn

n_classes = 4 ## Number of classes in our rice dataset

n_inputs = model.classifier[-1].in_features
last_layer = nn.Linear(n_inputs,n_classes)
model.classifier[-1] = last_layer
# if GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()
# check to see that your last layer produces the expected number of outputs
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=4, bias=True)
)


In [16]:
import torch.optim as optim
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [17]:
# number of epochs to train the model
n_epochs = 30

print ("Start training")

for epoch in range(1, n_epochs+1):
    
    train (model, device, train_loader, optimizer, criterion, epoch)
    test (model, device, criterion, test_loader)
    
    print ('\n')
            
print ("Done training")

Start training
Epoch: 1 , Training Accuracy: 146/348 (42%) Training Loss: 0.014838
Test loss: 0.0142, Test Accuracy: 37/80 (46%)



Epoch: 2 , Training Accuracy: 232/348 (67%) Training Loss: 0.009272
Test loss: 0.0122, Test Accuracy: 51/80 (64%)



Epoch: 3 , Training Accuracy: 268/348 (77%) Training Loss: 0.007427
Test loss: 0.0092, Test Accuracy: 62/80 (78%)



Epoch: 4 , Training Accuracy: 280/348 (80%) Training Loss: 0.005683
Test loss: 0.0083, Test Accuracy: 62/80 (78%)



Epoch: 5 , Training Accuracy: 295/348 (85%) Training Loss: 0.005058
Test loss: 0.0083, Test Accuracy: 63/80 (79%)



Epoch: 6 , Training Accuracy: 307/348 (88%) Training Loss: 0.003712
Test loss: 0.0079, Test Accuracy: 63/80 (79%)



Epoch: 7 , Training Accuracy: 308/348 (89%) Training Loss: 0.003235
Test loss: 0.0078, Test Accuracy: 62/80 (78%)



Epoch: 8 , Training Accuracy: 321/348 (92%) Training Loss: 0.002074
Test loss: 0.0077, Test Accuracy: 65/80 (81%)



Epoch: 9 , Training Accuracy: 324/348 (93%) Train

In [18]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 81 %


In [19]:
torch.save(model.state_dict(), 'n3.pwf')